<a href="https://colab.research.google.com/github/Minakshi654/Modelname/blob/main/Predicition_of_Data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In summary, a lower MSE is better because it means the model’s predictions are more accurate. Based on your results, RandomForest and DecisionTree are the best models for predicting whether the ‘Max Total’ will overflow in your dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import datetime

# Load the dataset
df = pd.read_csv('grouped_intervals.csv')

# Preprocess the data
df['Interval Start'] = pd.to_datetime(df['Interval Start'])
df['Day'] = df['Interval Start'].dt.dayofyear

# Features and target variable
X = df[['Day']]
y = df['Max Total']
#Handle missing values in 'y' - either remove or impute
# Remove rows with missing 'Max Total' values
df.dropna(subset=['Max Total'], inplace=True)

# Recalculate features and target after handling missing data
X = df[['Day']]
y = df['Max Total']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
models = {
    'RandomForest': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(),
    'SVR': SVR(),
    'KNeighbors': KNeighborsRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'MLP': MLPRegressor(max_iter=500)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name} MSE: {mse}")

# Use the RandomForest model to predict for the upcoming year
future_dates = pd.date_range(start="2024-08-01", end="2025-07-31", freq='D')
# future_dates = pd.date_range(start="2024-08-01", end="2024-08-31", freq='D')
future_days = future_dates.dayofyear
future_X = pd.DataFrame(future_days, columns=['Day'])

rf_model = models['RandomForest']
future_predictions = rf_model.predict(future_X)

# Create a DataFrame for the predictions
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted Max Total': future_predictions
})
predictions_df['Status'] = np.where(predictions_df['Predicted Max Total'] >= 1, 'Overflow', 'Normal')

# Save the predictions to a new CSV file
predictions_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")


RandomForest MSE: 0.02618346796407171
LinearRegression MSE: 0.03922805662664355
DecisionTree MSE: 0.026178824933861856
